<img src="https://snipboard.io/Kx6OAi.jpg">

# Session 3. Dask + Pandas
<div style="margin-top: -10px; padding: 5px; line-height: 20px;"><img src="https://snipboard.io/v5q47G.jpg" style="width: 35px; float: left; margin-right: 10px;"> Author:  <a href="http://www.linkedin.com/in/davidyerrington">David Yerrington</a>, Data Scientist<br>San Francisco, CA</div>

## Learning Objectives

- Be able to explain the difference between Pandas and Dask DataFrames
- Become familiar with storage options
- Understand nuances with schema

### Prerequisite Knowledge
- Basic Pandas 

## Environment Setup

We will first review some basic points to setup Python and the environment to start in [the setup guide](../environment.md).


# 1. Pandas and Dask

![](https://snipboard.io/QkGlLK.jpg)
> Photo by [VGC](https://www.facebook.com/vgcphotography/)

Many of us learn Pandas inside and out, staying completely in the safety of our Jupyter environments.  Most of what we need to deal with after we do analysis, engineer features, building cool recommendation prototypes, or supervised models means we're bound to leave the safety of Pandas for "real" big data frameworks like Spark.  This doesn't have to be the case anymore since Dask has reached a level of maturity that can reliably scale our work to production systems and beyond.

With a bit of knowledge around how DataFrames work in Dask, we can adapt existing work we've written in Pandas to Dask as long as we learn the basics.  Even though we're working on a single machine, there are still lots of performance gains by using a local cluster.

### In this session we will dive into:
- How to adapt common Pandas workflows to Dask
- Understand common pitfalls

## The Dask DataFrame

![](https://snipboard.io/CIHP8D.jpg)

So far we've looked at `Bag` and `Array`. Many of the same assumptions with those collection types apply to the Dask DataFrame which are:

- A Dask DataFrame is comprised of smaller DataFrames (partitions).
- Like `Array`, not every method is implemented 1:1.
- Column typing can be a bit difficult.
- Not every method behaves exactly the same.
- Joins are expensive when you don't use an index.
- Operations like `.set_index`, `.melt`, `.stack`/`.unstack`, `groupby().apply()` can be a bit on the slow side.

To master Dask DataFrames takes time.  There are tons of rules of thumb that are good to know but come with experience.  This is an intro-level session but we will explore the areas of Pandas-to-Dask that will give you the most impact and equip you with enough foundational knowledge to continue learning.

### Imports

In [ ]:
import dask
from dask.distributed import Client

client = Client()
client

### Make Some "Fake" People Data

The datasets `make_people` function practically puts our dataset into a `Bag` object for us.  It's only semi-structured once we convert it to a Dask `DataFrame`.

In [ ]:
people = dask.datasets.make_people().to_dataframe()

### Examining our DataFrame

Out of the box, the first problem we will find when examining data is that fact that we can't see any of it.

### `.info` isn't as informative at first glance

One of the first things we should be doing is looking for missing values and expected types.

### Question:  Any ideas on how to know what's missing and if the dtypes are expected?
Follow up: Can we trust the `dtypes` reported?  Why?

>  Quick refresher:  `.info()` with regular Pandas tells us the  name of each variable, how many non-null records, size in memory, and how many records there are.

### Everything is lazy, including `.describe`

In our very specific case, we have a DataFrame from a `Bag`.  It will load fast.  However, if you are loading from a method like `dask.read_csv("people-*.csv")` where each file has the same columns, by the time you call `.compute()` it would have to read them all into memory and **then** perform the computations accross every dimension for many of the build-in methods such as `.describe`.  Keep in mind that pre-requisite steps loaded in the dag will need to be performed to read the data before it will be computed.

### `.compute()` gives you eveything

`.compute()` returns every observersion with specified on a Dask `DataFrame` with no operations before it.  The return is a Pandas `DataFrame`.

>  Be careful when returning everything.  `.compute()` because it may not fit in memory.

### `.head()` is still your friend
Head fortunately isn't too different but it is roughly the equivalent to `.compute()` since it does bring data into context.  The data returned however is no longer a Dask DataFrame, but a regular Pandas `DataFrame`.  

- `.head()` will `.compute()` but only return limited results.
- Only the first partition is pulled from unless specified with `npartitions`

> **If you had 10 records each in 10 partitions**
> - The maximum number of records `.head()` will return is 10 with `.head(npartitions=1)` (the default).
> - `.head(npartitions=2)` would return the maximum number of records in 2 partitions (ie: 20). 
>

### Get a specific partition with `.get_partition(n)`

If you need a specifc chunk, like `.head()`, `.get_partition(n)` is handy to know and works well with `.head()` or `.compute()`

> It's also "lazy".  Also, note the index doesn't change.

### Aggregates work great as well.

In [ ]:
# groupby

In [ ]:
# mean age

### `.map()` and `.apply()` are also still the same.

A common strategy is to test your map with a smaller subset `.head()`.

In [ ]:
# Basic map function

Then once you've got it right, assign an entirely new column.

In [ ]:
# Create new features from nested tuple features

### Our map and apply functions don't run until `.compute()` is run.

If you queue up a bunch of transformations, these will all be added the the DAG.

In [ ]:
%%time
#

### Persisting

If our dataset can fit into memory, we may want to use `.persist()`

In [ ]:
people = people.persist()

Persisting a portion in memory is also pretty handy for developing on a smaller scale to test your code before scaling up.  There's always a question to be asked "Do I need to keep working in Dask?"  Sometimes you just need a sample and then you can break off a piece and move back to Pandas.  That's totally fine too if you're just examining a portion of data for a while for instance looking at timeseries for a specific time period rather than the entire history of data.

In [ ]:
%%time
#

### Avoid calling `.compute()` too often.

Whenever you run a compute, remember it will carry all the other processes in the graph with it and can be expensive.

In [ ]:
# This will run the graph 2x


In [ ]:
# This will run it 1x -- combine tasks when possible


### Using Parquet Files

You may get to a point where lots of transformations have been performed such as when you have cleaned your data or engineered a bunch of features.  Rather than carry those tasks across the graph to continue working, you may want to save your data.

The standard formats that Pandas supports, Dask also supports for the most part.  A format that is good to know how to work with is Apache Parquet.

<center><img src="https://snipboard.io/q1Lx25.jpg"></center>

Parquet is:
- Cross-platform
- Fast
- Strongly typed
- [Columnar-based](https://www.techwell.com/techwell-insights/2020/05/benefits-using-columnar-storage-relational-database-management-systems)  
- Compatible with many big data systems
- Supports compression encodings

Also, Dasks implementation of Parquet supports storage_options for cloud storage such as `s3:// or `gcs://`:
```python
people.to_parquet(
    "s3://my_bucket/my_directory/people.parquet"
    storage_options = {
        'account_name': 'ACCOUNT_NAME',
        'account_key': 'ACCOUNT_KEY'
    }, 
    ...
)

```

>  Another good one is **Avro**.  Avro files also have a lot of the same benefits.  If you are on Google Cloud and load a lot of data to BigQuery, the last time I checked, you don't get charged for using the Avro format.  Choosing the right format for larger datasets depends on the use case.  Using Avro with `DataFrame` is a bit trickier unless you convert back to `Bag` format then define a schema with a JSON file.  You can read about it more in [the docs](https://docs.dask.org/en/latest/bag-creation.html).

## Saving to formats like Parquet or Avro requires you to be a bit more specific about your data.

One of the big problems you'll run into using these types of formats is that `object` isn't specific enough to allow saving to partuet format.  Parquet tries to save it as a UTF8 string but at the lowest level in the DataFrame, the `name` field is actually a `tuple`.

In [ ]:
# inspect name feature

In [ ]:
# attempt to save file and see error
people.to_parquet("../data/people.parquet", object_encoding='UTF8')

### Question:  What should we do?

In [ ]:
# Attempt something here and report in thread.

## One more thing:  Working with Pandas

So far we've looked into working with Dask DataFrames staying within the ecosystem of Dask.  In the real world you may start a project in Pandas and then find the scope of your projects creeping up on you.  Or you perhaps you have a [gridsearch problem](https://ml.dask.org/hyper-parameter-search.html) that has grown out of control with sklearn.  It's fairly straight forward to convert your Pandas DataFrames on over.

In [ ]:
import pandas as pd

pokemon = pd.read_csv("../data/pokemon.csv")

Using `dask.dataframe.from_pandas()` we have to define either `npartitions=n` or `chunksize=n`.

- `npartitions`, Dask will attempt to divide your rows equally amongst `n` specified parititons.  If we have 800 rows, and we say `npartitions=8` then it will create a Dask `DataFrame` with 8 partitions having 100 rows in each partition.
- `chunksize`:  Dask tries to create partitions equally based on how many rows you specify per partition.


> #### General rules of thumb with Pandas to -> Dask
>
> - Stay in Pandas unless you have a real need to scale.  Pandas is faster to program in but there are benefits to parallalizing your code for CPU intensive transformations.
> - For Partition size, try to evenly distribute your data so data is spread evenly across your workers.
> - Don't be afraid to convert back from Dask to Pandas for portions of your project when it makes sense.

## Scaling to Cloud

I'm sure someone will have asked about this by now.  For the most part we've explored the Dask framework for parallel computing.  There will be a time when you will need to setup a real cluster and move beyond your laptop.  My recommended options include:

### Kubernetes

Kubernetes is a framework for managing contianerized resources. There's a bit of a learning curve with Kubernetes but it's very robust in terms of booting up containers and even autoscaling based on usage.  THere are plenty of cloud providers that offer hosted Kubernetes solutions and the best one is not surprisingly Google (they make Kubernetes).  

[Kubernetes + Helm](https://docs.dask.org/en/latest/setup/kubernetes-helm.html)https://github.com/jmcarpenter2/swifter

### Yarn on AWS EMR or GCS Dataproc

Yarn is a cluster manager and both AWS and Google Cloud offer a hosted version of this well-supported service.  

[yarn.dask.org](https://yarn.dask.org/en/latest/)

### Dask Cloud Provider

The Dask cloud Provider library includes a handful of APIs for deploying directly to cloud-specific VMs.

[Dask Cloud Provider](https://cloudprovider.dask.org/en/latest/azure.html) 


### Saturncloud.io - Vendor specific

Saturncloud.io exclusively provides Dask services for your choice of cloud provider and has their own API for integration.  Their strength is in having a complete end-to-end data science set of services that include Jupyter, GPU-enabled hardware, and some 3rd-party integrations.  What I like about this service is that if you want to get started with a Jupyter instance and have it outfitted to a Dask cluster with minimal effort, this is a great option.

[Saturncloud.io](https://www.saturncloud.io/s/)

### Resources

- A handy [Dask cheatsheet](https://docs.dask.org/en/latest/cheatsheet.html) from Anaconda.
- A great but intense look at [optimizing graphs](https://docs.dask.org/en/latest/optimize.html) Dask guide. 
- Newer Gridsearch alternative "Hyperband" for [hyperparameter tuning](https://examples.dask.org/machine-learning/hyperparam-opt.html) with Dask.
- Using Dask Array to [scale Sklearn predictions](https://examples.dask.org/machine-learning/parallel-prediction.html).
- [Swifter](https://github.com/jmcarpenter2/swifter) is an interesting wrapper for Pandas that automatically loads Dask as an extension for quick and easy jobs.

# Summary

- A Dask DataFrame is comprised of smaller DataFrames (partitions).
- Like Array, not every method from Pandas is implemented in Dask DataFrames.

### Dask DataFrames as still lazy!

Nothing is computed until you ask.  

### `.compute()` and `.head()` return Pandas DataFrames

- `.compute()` will process all records through the graph.
- `.head()` will only process the first partition 

### Functions like `.info` and `.describe()` don't tell the whole story

With large datasets, these methods might actually be impractical to run but they are also "lazy."  It's still a good idea to use samples or smaller subsets.  Its not a bad idea to use `get_partition` with these to inspect a bit more granularly.

### Whenever you add an operation without `.compute`, it adds it to the graph

This is desirable because you want to avoid calling `.compute` once you're working on a large dataset.  So if you're loading a bunch of CSV files or files in sequence, any kind of cleaning operation you do on it prior to running say an aggregation on a column like `df['age'].max()`, Dask will execute the entire graph that includes any data loading and cleaning prior to getting to the aggregation.

### Use `.persist()` to incrementally store data in memory.

As you interatively update code and write your transformations, the persist function is handy to store a portion or all of your data in memory when possible.

### Formats for storing files like Parquet are a bit more specific.

For literal Python objects and JSON, you will have to convert them explicity to strings, even if they are typed as an `object` in the DataFrame before storage. This type of format enforces a stricter control over what format of data it expects.  However, you do gain the potential to save your data to different cloud bucket services like S3.